In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import h5py
import matplotlib as mpl


from holodeck import plot, detstats
import holodeck.single_sources as sings
from holodeck.constants import YR, MSOL, MPC
import holodeck as holo

import hasasia.sim as hsim


In [ ]:
SHAPE = None
NREALS = 30
NFREQS = 40
NLOUDEST = 10

CONSTRUCT = False
JUST_DETSTATS = False
SAVEFIG = True
TOL=0.01
MAXBADS=5

NVARS = 21

NPSRS = 40
NSKIES = 25
TARGET = 'hard_time' # EDIT AS NEEDED
TITLE = '$\\tau_\mathrm{hard}$'   # EDIT AS NEEDED

In [ ]:
path = '/Users/emigardiner/GWs/holodeck/ecg-notebooks/parameter_investigation/anatomy_09B'
load_data_from_file = path+f'/{TARGET}_{NVARS}vars_clbrt_pta.npz' 
save_data_to_file = path+f'/{TARGET}_{NVARS}vars_clbrt_pta.npz' 

if CONSTRUCT is False:
    import os
    if os.path.exists(load_data_from_file) is False:
        err = 'load data file does not exist, you need to construct it.'
        raise Exception(err)
print(load_data_from_file)
print(save_data_to_file)

# Construct Parameter Space

In [ ]:
pspace = holo.param_spaces.PS_Uniform_09B(holo.log, nsamples=1, sam_shape=SHAPE, seed=None)
param_names = pspace.param_names
num_pars = len(param_names)
pars = 0.5*np.ones(num_pars)

In [ ]:
def vary_parameter(
        target_param,    # the name of the parameter, has to exist in `param_names`
        params_list = [0.0,  0.5, 1.0],  # the values we'll check
        pspace = holo.param_spaces.PS_Uniform_09B(holo.log, nsamples=1, sam_shape=SHAPE, seed=None),
        nreals=NREALS, nfreqs=NFREQS,
        pars=None, save_dir=None, 
        ):
    # get the parameter names from this library-space
    param_names = pspace.param_names
    num_pars = len(pspace.param_names)
    print(f"{num_pars=} :: {param_names=}")

    # choose each parameter to be half-way across the range provided by the library
    if pars is None:
        pars = 0.5 * np.ones(num_pars) 
    str_pars = str(pars).replace(" ", "_").replace("[", "").replace("]", "")
    # Choose parameter to vary
    param_idx = param_names.index(target_param)

    data = []
    params = []
    for ii, par in enumerate(params_list):
        pars[param_idx] = par
        print(f"{ii=}, {pars=}")
        # _params = pspace.param_samples[0]*pars
        _params = pspace.normalized_params(pars)
        params.append(_params)
        # construct `sam` and `hard` instances based on these parameters
        sam, hard = pspace.model_for_params(_params, pspace.sam_shape)
        if isinstance(hard, holo.hardening.Fixed_Time_2PL_SAM):
            hard_name = 'Fixed Time'
        elif isinstance(hard, holo.hardening.Hard_GW):
            hard_name = 'GW Only'
        # run this model, retrieving binary parameters and the GWB
        _data = holo.librarian.run_model(sam, hard, nreals, nfreqs, nloudest=NLOUDEST,
                                        gwb_flag=False, singles_flag=True, params_flag=True, details_flag=True)
        data.append(_data)
    if save_dir is not None:
        str_shape = str(sam.shape).replace(", ", "_").replace("(", "").replace(")", "")
        filename = save_dir+'/%s_p%s_s%s.npz' % (target_param, str_pars, str_shape)
        np.savez(filename, data=data, params=params, hard_name=hard_name, shape=sam.shape, target_param=target_param )
        print('saved to %s' % filename)

    return (data, params)

## Calculate Data (if CONSTRUCT) and Individual DetStats (if CONSTRUCT or JUST_DETSTAATS)

In [ ]:
if JUST_DETSTATS or CONSTRUCT:
    # get data (not dsdat_)
    if CONSTRUCT:
        params_list = np.linspace(0,1,NVARS)
        data, params, = vary_parameter(target_param=TARGET, params_list=params_list)
    else:
        file = np.load(load_data_from_file, allow_pickle=True)
        print(file.files)
        data = file['data']
        params = file['params']
        file.close()

    fobs_cents = data[1]['fobs_cents'] 
    dur = 1.0/fobs_cents[0]
    cad = 1.0/(2*fobs_cents[-1])
    
    # get dsdat for each data/param
    dsdat = []
    for ii, _data in enumerate(data):
        print(f"{ii=}")
        # get strain info
        hc_bg = _data['hc_bg']
        hc_ss = _data['hc_ss']
        _dsdat = detstats.detect_pspace_model_clbrt_pta(fobs_cents, hc_ss, hc_bg, NPSRS, NSKIES, debug=True, tol=TOL, maxbads=MAXBADS)
        dsdat.append(_dsdat)
        
    np.savez(save_data_to_file,
             data = data, dsdat=dsdat, params=params)
else:
    file = np.load(load_data_from_file, allow_pickle=True)
    print(file.files)
    data = file['data']
    params = file['params']
    dsdat = file['dsdat']
    file.close()


# Plot

In [ ]:

def draw_skies_vs_bg(ax, skies_ss, dp_bg, label=None,
                     color='k', mean=True):
    xx = dp_bg # shape (R,)
    if mean:
        yy = np.mean(skies_ss, axis=-1) # shape (R,)
    else: 
        yy = np.mean(skies_ss, axis=-1) # shape (R,)
    yerr = np.std(skies_ss, axis=-1) # shape (R,)

    hh = ax.errorbar(xx, yy, yerr, color=color, label=label,
                linestyle='', capsize=3, marker='o', alpha=0.5)
    return hh

def plot_dpss_vs_dpbg(dsdat, params, target_param=TARGET, use_ev=True, title=TITLE):
    colors = cm.rainbow_r(np.linspace(0, 1, len(dsdat)))
    sslabel = 'Expected Number' if use_ev else 'Detection Probability'
    fig, ax = plot.figax(xlabel='Background Detection Probability', 
                         ylabel='$\langle$ Single Source %s} $\\rangle_\mathrm{skies}$' % sslabel,
    ) #xscale='linear', yscale='linear')
    handles = []
    for ii, ds in enumerate(dsdat):
        label = '%.2f' % params[ii][target_param]
        detss = ds['ev_ss'] if use_ev else ds['dp_ss']
        hh = draw_skies_vs_bg(ax, detss, ds['dp_bg'], color=colors[ii], label=label)
        handles.append(hh)
    ax.legend(handles=handles, loc = 'lower left', title=title,
              ncols=4, title_fontsize=12)

    fig.tight_layout()
    return fig

text = 'GSMF: $\psi_0=%.2f, m_{\phi,0}=%.2f$' % ((params[0]['gsmf_phi0']), (params[0]['gsmf_mchar0_log10']))
text = text+'\nMMB: $\mu = %.2f, \epsilon_\mu=%2f$ dex' % (params[0]['mmb_mamp_log10'], params[0]['mmb_scatter_dex'])
text = text+'\n$da/dt: \gamma_\mathrm{inner}=%.2f, \\tau_\mathrm{hard}=%.2f$' % (params[0]['hard_time'], 2.5)
print(text)




fig1 = plot_dpss_vs_dpbg(dsdat, params, use_ev=True)
fig1.axes[0].text(0.99,0.01, text, transform=fig1.axes[0].transAxes, verticalalignment='bottom', horizontalalignment='right')
fig1.tight_layout()
if SAVEFIG: 
    param_path = '/Users/emigardiner/GWs/holodeck/output/figures/params'
    fig1.savefig(param_path+f'/ss_vs_bg_{TARGET}_{NVARS}vars_clbrtd.png')


fig2 = plot_dpss_vs_dpbg(dsdat, params, use_ev=False)
fig2.axes[0].text(0.99,0.99, text, transform=fig1.axes[0].transAxes, verticalalignment='top', horizontalalignment='right')
fig2.tight_layout()

In [ ]:

# xlabel=TITLE
# ylabel='$\gamma_\mathrm{SS}/\gamma_\mathrm{BG}$'
# colors = cm.rainbow_r(np.linspace(0, 1, len(params)))


# fig, ax = plot.figax(xscale='linear',)
# ax.set_ylabel(ylabel, fontsize=14)
# ax.set_xlabel(xlabel, fontsize=14)

# target=TARGET
# xx = []
# yy = []
# for ii, par in enumerate(params):
#     xx.append(params[ii][target])
#     dp_bg = dsdat[ii]['dp_bg']
#     dp_ss = np.mean(dsdat[ii]['dp_ss'], axis=-1)
#     yy.append(dp_ss/dp_bg)
#     for rr in range(len(yy[0])):
#         ax.scatter(xx[ii], yy[ii][rr], color=colors[ii], alpha=0.5)
# xx = np.array(xx)
# yy = np.array(yy)
# print(yy.shape)
# # for rr, yi in enumerate(yy):
# #     ax.scatter(xx,yy[:,rr])
# plot.draw_med_conf(ax, xx, yy)
# # for rr in range(len(yy[0])):
# #     ax.scatter(xx,yy[:,rr], alpha=0.5, color=colors[])

Could do the same, but vary another model parameter for color, or show some other parameter

# Get Mass Normalization

In [ ]:
# # Get SS mass normalization
# use_snr=False
# # ssmtot = []
# # ss_dp = []
# if use_snr:
#     mt_hisnr= np.zeros((NVARS, NREALS))
#     for ii,dat in enumerate(data):
#         # ssmtot.append(dat['sspar'][0])
#         snr_ss = dsdat[ii]['snr_ss'] # (F,R,S,L)
#         # print(snr_ss.shape)
#         ssmtot = dat['sspar'][0]
#         # print(ssmtot.shape)
#         # for each realization, find the frequency with the loudest single source
#         for rr in range(len(snr_ss[0])):
#             argmax = np.argmax(snr_ss[:,rr,:,:])
#             fidx, sidx, lidx = np.unravel_index(argmax, (NFREQS, NSKIES, NLOUDEST))
#             # print(f"{fidx=}, {rr=}, {sidx=}, {lidx=}")
#             mt_hisnr[ii,rr] = ssmtot[fidx, rr, lidx]

#     mt_hisnr = np.log10(mt_hisnr/MSOL)
#     vmin = np.min(mt_hisnr)
#     vmax = np.max(mt_hisnr)
#     norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)

# # print(f"{vmin=}, {vmax=}")
# # print(mt_hisnr.shape)

# Any Normalization

In [ ]:

#### use mass of max-DP single source for each realization
norm_by_maxDPmass = True
if norm_by_maxDPmass:
    normcol = np.zeros((NVARS, NREALS))
    for ii, dat in enumerate(data):
        gamma_ssi = dsdat[ii]['gamma_ssi'] # (F,R,S,L)
        ssmtot = dat['sspar'][0] # (F,R,L)
        # for each realization, find the frequency, sky, and loudest with the loudest single source
        for rr in range(NREALS):
            argmax = np.argmax(gamma_ssi[:,rr,:,:])
            ff, ss, ll = np.unravel_index(argmax, (NFREQS, NSKIES, NLOUDEST))
            normcol[ii,rr] = ssmtot[ff,rr,ll]
    normcol = np.log10(normcol/MSOL)

#### could include other colorbar options here, to set normcol values

norm = mpl.colors.Normalize(vmin=np.min(normcol), vmax=np.max(normcol))

# Plot with mass colorbar

In [ ]:
xlabel=TITLE
# ylabel='$\gamma_\mathrm{SS}/\gamma_\mathrm{BG}$'
ylabel = '$EV_\mathrm{SS} / DP_\mathrm{BG}$'
cmap = cm.rainbow

fig, ax = plot.figax(xscale='linear',)
ax.set_ylabel(ylabel, fontsize=14)
ax.set_xlabel(xlabel, fontsize=14)

target=TARGET
xx = []
yy = []
for ii, par in enumerate(params):
    xx.append(params[ii][target])
    dp_bg = dsdat[ii]['dp_bg']
    dp_ss = np.mean(dsdat[ii]['ev_ss'], axis=-1)
    yy.append(dp_ss/dp_bg)
    for rr in range(len(yy[0])):
        ax.scatter(xx[ii], yy[ii][rr], color=cmap(norm(normcol[ii,rr])),  alpha=0.5,)
xx = np.array(xx)
yy = np.array(yy)
print(yy.shape)
plot.draw_med_conf(ax, xx, yy)
cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap), label='log ($M(\mathrm{max\ EV_\mathrm{SS}}) / [\mathrm{M}_\odot]$)',)
fig.tight_layout()

figloc = '/Users/emigardiner/GWs/holodeck/output/figures/params'
if SAVEFIG: fig.savefig(figloc+'/dpratio_vs_%s_w_mass_%dvars_clbrtd.png' % (TARGET, NVARS))